## 1. Import packages

In [1]:
import glob
import time
from PIL import Image
import torch
from tqdm import tqdm
from ultralytics import YOLO
import sys
sys.path.append("..")
from app.utils.process import preprocess_image

## 2. Config

In [2]:
ONNX_PATH = "/home/mlops/Repository/aio2025-onnx-tensorrt/models/yolo26l.onnx"
IMAGE_GLOB = "/home/mlops/Repository/aio2025-onnx-tensorrt/images/val2014/*.jpg"
IMG_SIZE = 640
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
WARMUP = 20
SAMPLE_SIZE = 1000

In [3]:
# Load images
image_paths = sorted(glob.glob(IMAGE_GLOB))
assert len(image_paths) > 0, "No images found"
if SAMPLE_SIZE != -1:
    image_paths = image_paths[:SAMPLE_SIZE]

## 3. Model

In [4]:
# Load model
model = YOLO(ONNX_PATH, task='detect')

# Warmup
dummy = torch.zeros((1, 3, IMG_SIZE, IMG_SIZE), device=DEVICE)
for _ in range(WARMUP):
    _ = model(dummy, verbose=False)

Loading /home/mlops/Repository/aio2025-onnx-tensorrt/models/yolo26l.onnx for ONNX Runtime inference...


Using ONNX Runtime 1.21.0 with CUDAExecutionProvider


2026-01-16 16:47:24.712174957 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 6 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.


## 4. Test loop

In [5]:
# FPS test
start = time.perf_counter()

for p in tqdm(image_paths):
    img = Image.open(p)
    processed_image, _ = preprocess_image(img, IMG_SIZE)
    _ = model(processed_image, verbose=False)
end = time.perf_counter()

total_time = end - start
fps = len(image_paths) / total_time

100%|██████████| 1000/1000 [00:28<00:00, 34.70it/s]


In [6]:
print("=" * 40)
print(f"Device           : {DEVICE}")
print(f"Images processed : {len(image_paths)}")
print(f"Total time       : {total_time:.2f} s")
print(f"FPS              : {fps:.2f}")
print("=" * 40)

Device           : cuda
Images processed : 1000
Total time       : 28.82 s
FPS              : 34.70
